## 로지스틱 회귀분석

1. Logistic Regression 기본 배경

    - 참/거짓으로 분류

    - 이분으로 분류 : 합격인가 불합격인가
    
    - 로지스틱 회귀 분석은 연속적인 값을 가지더라고, 로지스틱 함수의 결과값은 0과 1 사이의 값을 갖기에 이분법적인 분류 문제 해결하는 모형으로 적합하다
    
    - 이름은 회귀이지만 분류 모형이다
    

2. LOGIT 적용분야
    - 기업 부도 예측
    - 주가-환율-금리 예측
    
[참고] [로지스틱 회귀](https://thebook.io/080228/part02/ch05/01/)
<img src='./imgs/logistic1.png' width='500'>
<img src='./imgs/logistic2.png' width='500'>


[참고] [시그모이드 함수](https://thebook.io/080228/part01/ch02/06/)

[참고] [로지스틱회귀에서 퍼셉트론으로](https://thebook.io/080228/part02/ch05/)
<img src='./imgs/logistic3.png' width='500'>
<img src='./imgs/logistic4.png' width='400'>

[그림] 모두의 딥러딩

이 퍼셉트론은 그 후 인경신경망, 오차역전파 등의 발전을 거쳐 딥러닝으로 이어지게 된다.

#### 타이타닉 데이타

성별(Sex), 나이(Age), 객실등급(Pclass), 요금(Fare)가 생존에 어느 정도의 영향을 미치는가

In [1]:
import pandas as pd
import numpy as np

# 데이타 로딩
df = pd.read_csv('../data/titanic/temp/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


데이타처리

    1) 나이, 요금은 숫자형으로 그래도 사용

    2) 성별과 male/ female로 바로 처리할 수 없다 

    3) 객실등급은 1, 2, 3 으로 이루어져 있지만, 3등석이 높은 값으로 처리하여 결과가 외곡될 수 있다.

### 데이터 구조  만들기

In [3]:
cols_to_keep = ['Survived','Age','Fare']


# one hot incoding 방식
dummy_Pclass=pd.get_dummies(df["Pclass"],prefix="Pclass")
dummy_Pclass





,Pclass_1,Pclass_2,Pclass_3
0,False,False,True
1,True,False,False
2,False,False,True
3,True,False,False
4,False,False,True
...,...,...,...
886,False,True,False
887,True,False,False
888,False,False,True
889,True,False,False


컬럼을 거짓과 진실로 두가지로 나누어서 판별하기 쉽게 변형

In [6]:
#성별에 대한 데이터 처리

dummy_Sex=pd.get_dummies(df["Sex"],prefix="Sex") #Prefix로 변수를 지정을 해주지 않으면 숫자값이 나온다.
dummy_Sex
#값을 구분하기 쉽게끔

,Sex_female,Sex_male
0,False,True
1,True,False
2,True,False
3,True,False
4,False,True
...,...,...
886,False,True
887,True,False
888,True,False
889,False,True


이렇게 나누어 주면 남자일때 True false일때의 각각의 경우를 나누어준다.

In [8]:
data=df[cols_to_keep] #기존에 필요한 컬럼들을 미리 변수에 저장을 한것들을 넣어준다.

#df[['Survived','Age','Fare']] #주의 이렇게도 가능하지만 바깥은 판다스에 있는 데이터 프레임의 괄호 안쪽은 1차원리스트배열의 괄호

data


data = data.join(dummy_Sex)

data = data.join(dummy_Pclass)
#조인을 해준 결과를 변수에 담아줘야한다.
data


,Survived,Age,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,0,22.0,7.2500,False,True,False,False,True
1,1,38.0,71.2833,True,False,True,False,False
2,1,26.0,7.9250,True,False,False,False,True
3,1,35.0,53.1000,True,False,True,False,False
4,0,35.0,8.0500,False,True,False,False,True
...,...,...,...,...,...,...,...,...
886,0,27.0,13.0000,False,True,False,True,False
887,1,19.0,30.0000,True,False,True,False,False
888,0,30.0,23.4500,True,False,False,False,True
889,1,26.0,30.0000,False,True,True,False,False


### (1) 데이터셋과 레이블(답)

In [9]:
train_lable = data["Survived"]
train_data = data[data.columns[1:]]

In [10]:
train_lable[0]

0

In [12]:
train_data.iloc[0]

Age            22.0
Fare           7.25
Sex_female    False
Sex_male       True
Pclass_1      False
Pclass_2      False
Pclass_3       True
Name: 0, dtype: object

0번째에 있는 사람은 남성에 3번 객실 사람에 22살인데 사망

### (2) 모델을 선택하여 학습

In [13]:
from sklearn.linear_model import LogisticRegression

In [15]:
lr = LogisticRegression()

lr.fit(train_data,train_lable)

LogisticRegression()

어떤 무언가의 규칙을 하나라도 얻었을 것이다.

### (3) 검증하기  
검증 하는 것도 직접해야하지만 여기서 프로그램한테 간단하게 맡기자

In [20]:
print("훈련 점수 :{:.2f}".format(lr.score(train_data,train_lable)))

훈련 점수 :0.80


80퍼 정도의 정확도가 있는 모델 생성

### (5)기존 전체 데이타로 예측하기

In [24]:
data['predict']=lr.predict(train_data)
data.head(20)

,Survived,Age,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,predict
0,0,22.0,7.2500,False,True,False,False,True,0
1,1,38.0,71.2833,True,False,True,False,False,1
2,1,26.0,7.9250,True,False,False,False,True,1
3,1,35.0,53.1000,True,False,True,False,False,1
4,0,35.0,8.0500,False,True,False,False,True,0
5,0,30.0,8.4583,False,True,False,False,True,0
6,0,54.0,51.8625,False,True,True,False,False,0
7,0,2.0,21.0750,False,True,False,False,True,0
8,1,27.0,11.1333,True,False,False,False,True,1
9,1,14.0,30.0708,True,False,False,True,False,1


#### 분석결과

* coef(회귀계수)를 기준으로 생존 확인

* coef 수가 클수록 비례하고 음의 값이면 반비례하는 것이다.


    - 나이가 적을 수록 생존확률이 높다
    - 요금이 높을 수록 생존확률이 높다
    - 객실등급에 따라 1등급 > 2등급 > 3긍급 순으로 생존확률이 높다
    - 여자가 남자보다 생존 확룔이 높다 ( Sex_male은 남성값이기에 음수는 여성이 생존확률이 높다는 것이다 )

* 객실등급과 성별이 생존에 많은 영향을 미쳤다


In [25]:
print("Age	Fare	Sex_female	Sex_male	Pclass_1	Pclass_2	Pclass_3	")
print(lr.coef_)

Age	Fare	Sex_female	Sex_male	Pclass_1	Pclass_2	Pclass_3	
[[-3.19959130e-02  1.08306305e-03  1.27114617e+00 -1.27412579e+00
   1.06182543e+00  4.94196518e-02 -1.11422470e+00]]


Survived 와 prdeict를 비교한다.

- Survived=1이면 생존이고 predict 0.5 이상의 값이면 생존으로 예측한 것이다.

- Survived=0이면 사망이고 predict 0.5 미만의 값이면 사망으로 예측한 것이다.

- 결과 확인 중 잘못 예측한 것도 있다 ( 14번째 )